# Chapter 13 Best Practices for the Real World

## 13.1 Getting the most out of your models

### 13.1.1 Hypterparameter optimization

+ The process of optimizing hyperparameters typically looks like this:
    1. Choose a set of hyperparameters (automatically).

    2. Build the corresponding model.
    
    3. Fit it to your training data, and measure performance on the validation data.
    
    4. Choose the next set of hyperparameters to try (automatically).
    
    5. Repeat.
    
    6. Eventually, measure performance on your test data.

#### Keras TUNER 

+ KerasTuner lets you replace hard-coded hyperparameter values, such as units=32,
with a range of possible choices, such as Int(name="units", min_value=16,
max_value=64, step=16). 

+ This set of choices in a given model is called the search space
of the hyperparameter tuning process.

pip install keras-tuner

To specify a search space, define a model-building function (see the next listing).
It takes an hp argument, from which you can sample hyperparameter ranges, and it
returns a compiled Keras model.

Listing 13.1 A KerasTuner model-building function

In [7]:
from tensorflow import keras
from keras import layers

from kerastuner import HyperParameters as hp



def build_model(hp):

#     Sample hyperparameter values from the
#  hp object. 
# After sampling, these values
#  (such as the "units" variable here) are
#  just regular Python constants

    units = hp.Int(name="units",min_value=16,max_value =64,step=16)

    model = keras.Sequential([
        layers.Dense(units=units,activation='relu'),
        layers.Dense(10,activation='softmax'),
    ])

    optimizer = hp.Choice(name = "optimizer", values = ["rmsprop","adam"])

    model.compile(optimizer = optimizer,
                    loss = keras.losses.SparseCategoricalCrossentropy(),
                    metrics = ['accuracy']
    )

    return model 
    

Listing 13.2 A KerasTuner HyperModel

In [8]:
import kerastuner as kt


class SimpleMLP(kt.HyperModel):
    def __init__(self,num_classes):
        self.num_classes = num_classes
    
    def build(self,hp):
        units  = hp.Int(name= "units",min_value = 16,max_value = 64,step = 16)
        model = keras.Sequential([
            layers.Dense(units=units,activation="relu"),
            layers.Dense(self.num_classes,activation='softmax')
        ])
        optimizer = hp.Choice(name= "optimizer",values =['rmsprop','adam'])
        model.compile(
            optimizer = optimizer,
            loss = keras.losses.SparseCategoricalCrossentropy(),
            metrics = ['accuracy']
        )

        return model

    

In [9]:
hypermodel = SimpleMLP(num_classes=10)

The next step is to define a __“tuner.”__ Schematically, you can think of a tuner as a for
loop that will repeatedly

1. Pick a set of hyperparameter values

2. Call the model-building function with these values to create a model

3. Train the model and record its metrics

+ KerasTuner has several built-in tuners available: $RandomSearch$ , $BayesianOptimization$, and $Hyperband$. 


+ Let’s try $BayesianOptimization$, a tuner that attempts to make smart predictions for which new hyperparameter values are likely to perform best
 given the outcomes of previous choices

In [12]:
from kerastuner import BayesianOptimization

tuner = BayesianOptimization(
    hypermodel=build_model,

#     Specify the metric that the tuner will seek to 
# optimize. Always specify validation metrics, 
# since the goal of the search process is to 
# find models that generalize!
    objective='val_accuracy',

#     Maximum number of different 
# model configurations (“trials”) 
# to try before ending the search.
    max_trials=100,
    
#     To reduce metrics variance, you can train the 
#  same model multiple times and average the results. 
#  executions_per_trial is how many training rounds 
#  _ _ (executions) to run for each model configuration (trial).
    executions_per_trial = 2,

   directory = "E:\\Python-Machine-Learning\\Deep_Learning_With_python\\Ch13\\mnist_kt_test",

#    Whether to overwrite data in directory to start a new search. Set this to 
# True if you’ve modified the model-building function, or to False to resume 
# a previously started search with the same model-building function.
   overwrite = True )

In [13]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


#### Objective Maximization and Minimization

For built-in metrics (like accuracy, in our case), the direction of the metric (accuracy
 should be maximized, but a loss should be minimized) is inferred by KerasTuner.
 However, for a custom metric, you should specify it yourself, like this:

 objective = kt.Objective(
  name="val_accuracy", 
  direction="max")

Finally, let's launch the search.

In [16]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train= x_train.reshape((-1,28*28)).astype("float32")/255
x_test= x_test.reshape((-1,28*28)).astype("float32")/255


x_train_full = x_train[:]
y_train_full = y_train[:]
x_test_full = x_test[:]

num_val_samples = 10000

x_train,x_val = x_train[num_val_samples:],x_train[:num_val_samples]
y_train,y_val = y_train[num_val_samples:],y_train[:num_val_samples]

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        patience = 5,
    )
]

In [19]:
# tuner.search(x_train,y_train,
#                 batch_size = 128,
#                 epochs = 30,
#                 validation_data = (x_val,y_val),
#                 callbacks = callbacks,
#                 verbose =0)

However, with a typical search
 space and dataset, you’ll often find yourself letting the hyperparameter search run
 overnight or even over several days. 
 
 If your search process crashes, you can always
 __restart__ it—just __specify overwrite=False__ in the tuner so that it can resume from the
 trial logs stored on disk.

Listing 13.3 Querying the best hyperparameter configurations

In [20]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

The one last parameter to be decided is the __epochs__.

Using an aggressive value of __patience__ in the __EarlyStopping()__ saves the best epoch, but it may lead to a __underfit__ model

#### An example of obtain best epochs

In [21]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            mode = 'min',
            ## Note the very high patience value
            patience =10
        )
    ]

    history = model.fit(
        x_train,y_train,
        validation_data  = (x_val,y_val),
        epochs = 100,
        batch_size =128,
        callbacks = callbacks,
        
    )

    val_loss_per_epoch = history.history['val_loss']

    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch))

In [22]:
def get_best_trained_model(hp):
    model  = build_model(hp)
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full,y_train_full,batch_size = 128,
        epoch = int(best_epoch*1.2)
    
    )
    return model

In [23]:
# best_models = []

# for hp in best_hps: 
#     model = get_best_trained_model(hp)
#     model.evaluate(x_test,y_test)
#     best_models.append(model)

Note that if you’re not worried about slightly underperforming, there’s a shortcut you
 can take: 
 
 just use the tuner to reload the top-performing models with the best weights
 saved during the hyperparameter search, without retraining new models from scratch

In [24]:
# best_models = tuner.get_best_models(top_n)

__Note__ that an important issue to think about is when doing automatic hyperparameter optimization at scale is __validation-set__ overfitting